## 환경 설정 (API KEY, langsmith, db)

In [1]:
## 환경
from dotenv import load_dotenv
import warnings
import os

warnings.filterwarnings("ignore")
load_dotenv()

True

In [ ]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다. 추적 안 하고싶으면 set_enalbel =Fasle로 설정
logging.langsmith("wiset-final")

In [3]:
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine, inspect, text
import os

db_path = os.getenv("DB_PATH")
if not os.path.exists(db_path):
    raise Exception("데이터베이스 파일이 존재하지 않습니다.")

db_uri = f"sqlite:///{db_path}"

# 엔진 생성
engine = create_engine(db_uri)

# 엔진 생성
engine = create_engine(db_uri)

# SQLDatabase 인스턴스 생성 시 엔진 객체 전달
db = SQLDatabase(engine)

In [4]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate

# 사용 가능한 테이블 이름 출력
print(db.get_usable_table_names())

print(db.dialect)

['Categories', 'CustomerCustomerDemo', 'CustomerDemographics', 'Customers', 'EmployeeTerritories', 'Employees', 'Order Details', 'Orders', 'Products', 'Regions', 'Shippers', 'Suppliers', 'Territories']
sqlite


## Cahin

In [5]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Based on the table schema below, write a SQL query that wold answer the user's question:
{schema}

Question: {question}
SQL Query
"""

prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['question', 'schema'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'schema'], template="\nBased on the table schema below, write a SQL query that wold answer the user's question:\n{schema}\n\nQuestion: {question}\nSQL Query\n"))]


In [6]:
def get_schema(_):
    return db.get_table_info()

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [8]:
llm = ChatOpenAI()

sql_chain =(
    RunnablePassthrough.assign(schema=get_schema)
    | prompt 
    | llm.bind(stop="\nSQL Result:")
    | StrOutputParser()
)

In [9]:
sql_chain.invoke({"question": "How may Employees are there?"})


'SELECT COUNT(EmployeeID) AS TotalEmployees\nFROM Employees;'

In [10]:
template = """
Based on the table schema below, qustion, sql query and sql response, write a natural language response that would answer the user's question:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt = ChatPromptTemplate.from_template(template)


In [11]:
def run_query(query):
    return db.run(query)

In [12]:
run_query("SELECT COUNT(*) AS TotalEmployees FROM Employees;")

'[(9,)]'

In [13]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema = get_schema,
        #response = lambda variables: print(variables)
        response = lambda variables: run_query(variables["query"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
full_chain.invoke({"question": "How many Employees are there?"})

'There are a total of 9 employees in the company.'

## AGENT

In [48]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.agents.output_parsers import ReActSingleInputOutputParser


# llm 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True)

# Create SQL toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

#  prompt 템플릿
custom_prompt = PromptTemplate.from_template(
    """You are an AI assistant for querying {dialect} databases. Answer the following question by creating and running SQL queries.

    Available tools:
    {tools}

    Use the following format:

    Question: {input}
    Thought: Consider what information is needed to answer the question
    Action: The action to take, should be one of {tool_names}
    Action Input: The input for the action
    Observation: The result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: The natural language response to the original input question

    {agent_scratchpad}

    Begin!

    Question: {input}
    Thought: """
)

# agent
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prompt=custom_prompt,
    handle_parsing_errors=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

# Run agent
response = agent_executor.invoke({"input": "직원은 몇명이야?"})["output"]
print(response)




> Entering new SQL Agent Executor chain...
To answer the question about the number of employees, I need to find the relevant table that contains employee information. 

Action: sql_db_list_tables  
Action Input: ""  Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, TerritoriesTo answer the question about the number of employees, I need to find the relevant table that contains employee information. The "Employees" table seems to be the one I need to query.

Action: sql_db_schema  
Action Input: Employees  
CREATE TABLE "Employees" (
	"EmployeeID" INTEGER, 
	"LastName" TEXT, 
	"FirstName" TEXT, 
	"Title" TEXT, 
	"TitleOfCourtesy" TEXT, 
	"BirthDate" DATE, 
	"HireDate" DATE, 
	"Address" TEXT, 
	"City" TEXT, 
	"Region" TEXT, 
	"PostalCode" TEXT, 
	"Country" TEXT, 
	"HomePhone" TEXT, 
	"Extension" TEXT, 
	"Photo" BLOB, 
	"Notes" TEXT, 
	"ReportsTo" INTEGER, 
	"PhotoPath" TEXT, 


In [50]:
# Run agent
response = agent_executor.invoke({"input": "구매가 많은 지역은?"})["output"]
print(response)



> Entering new SQL Agent Executor chain...
To determine the regions with the highest purchase volume, I need to identify the relevant tables that contain information about purchases and regions. 

Action: sql_db_list_tables  
Action Input: ""  Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, TerritoriesTo determine the regions with the highest purchase volume, I need to identify the relevant tables that contain information about purchases and regions. The "Orders" table likely contains purchase information, and the "Regions" table may provide the geographical context. I will check the schema of these tables to understand their structure.

Action: sql_db_schema  
Action Input: Orders, Regions  
CREATE TABLE "Orders" (
	"OrderID" INTEGER NOT NULL, 
	"CustomerID" TEXT, 
	"EmployeeID" INTEGER, 
	"OrderDate" DATETIME, 
	"RequiredDate" DATETIME, 
	"ShippedDate" DATETIME, 
	"Ship

In [51]:
agent_executor = create_sql_agent(
    llm,                    # 언어 모델 (Large Language Model)
    db=db,                  # 연결할 데이터베이스
    agent_type="openai-tools",  # agent 유형을 OpenAI 도구로 지정
    verbose=False           # 상세한 출력을 비활성화
)

In [52]:
response =  agent_executor.invoke("고객중에서 가장 구매를 많이한 top 10명과 각각 구매액을 표시해줘")["output"]
print(response)

Here are the top 10 customers with the highest purchase amounts along with their respective total purchase amounts:

1. **Océano Atlántico Ltda.** - $5,993,602.39
2. **Wellington Importadora** - $5,892,913.90
3. **Ottilies Käseladen** - $5,732,197.50
4. **Comércio Mineiro** - $5,570,956.91
5. **Save-a-lot Markets** - $5,566,881.45
6. **B's Beverages** - $5,509,214.47
7. **Wolski Zajazd** - $5,502,720.91
8. **Centro comercial Moctezuma** - $5,477,835.86
9. **Morgenstern Gesundkost** - $5,464,115.84
10. **Tradição Hipermercados** - $5,461,368.04

These customers have made the highest total purchases in the database.


In [53]:
query = """
SELECT 
    c.CustomerID, 
    c.CompanyName, 
    SUM(od.UnitPrice * od.Quantity) AS TotalPurchase
FROM 
    Customers c
JOIN 
    Orders o ON c.CustomerID = o.CustomerID
JOIN 
    "Order Details" od ON o.OrderID = od.OrderID
GROUP BY 
    c.CustomerID, c.CompanyName
ORDER BY 
    TotalPurchase DESC
LIMIT 10;
"""

result = db.run(query)
print(result)

[('OCEAN', 'Océano Atlántico Ltda.', 5993602.39), ('WELLI', 'Wellington Importadora', 5892913.9), ('OTTIK', 'Ottilies Käseladen', 5732197.5), ('COMMI', 'Comércio Mineiro', 5570956.91), ('SAVEA', 'Save-a-lot Markets', 5566881.45), ('BSBEV', "B's Beverages", 5509214.47), ('WOLZA', 'Wolski  Zajazd', 5502720.91), ('CENTC', 'Centro comercial Moctezuma', 5477835.86), ('MORGK', 'Morgenstern Gesundkost', 5464115.84), ('TRADH', 'Tradição Hipermercados', 5461368.04)]


In [54]:
response =  agent_executor.invoke("구매가 많은 지역은?")["output"]
print(response)

구매가 많은 지역은 다음과 같습니다:

1. Western Europe: 4971건
2. South America: 2881건
3. North America: 2835건
4. Southern Europe: 1695건
5. British Isles: 1378건
6. Central America: 838건
7. Northern Europe: 727건
8. Scandinavia: 515건
9. Eastern Europe: 194건
10. (정보 없음): 307건

가장 많은 구매가 이루어진 지역은 Western Europe입니다.


In [55]:
query = """
SELECT c.Region, COUNT(o.OrderID) AS OrderCount 
FROM Orders o 
JOIN Customers c ON o.CustomerID = c.CustomerID 
GROUP BY c.Region 
ORDER BY OrderCount DESC 
LIMIT 10;
"""

result = db.run(query)
print(result)

[('Western Europe', 4971), ('South America', 2881), ('North America', 2835), ('Southern Europe', 1695), ('British Isles', 1378), ('Central America', 838), ('Northern Europe', 727), ('Scandinavia', 515), (None, 307), ('Eastern Europe', 194)]
